----

### Task 1-3

In [6]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import time
from mpl_toolkits.mplot3d import Axes3D

Генетичний алгоритм (двійкове кодування)

In [ ]:
def genetic_algorithm(func, bounds, bits=16, pop_size=30, generations=80):
    a, b = bounds

    def decode(ind):
        return a + int("".join(map(str, ind)), 2) / (2**bits - 1) * (b - a)

    pop = np.random.randint(0, 2, (pop_size, bits))

    for _ in range(generations):

        fitness = np.array([func(decode(ind)) for ind in pop])

        idx1 = np.random.randint(0, pop_size, pop_size)
        idx2 = np.random.randint(0, pop_size, pop_size)

        mask = (fitness[idx1] < fitness[idx2])[:, None]
        pop = np.where(mask, pop[idx1], pop[idx2])

        # схрещування
        for i in range(0, pop_size-1, 2):
            if np.random.rand() < 0.8:
                pt = np.random.randint(1, bits)
                pop[i, pt:], pop[i+1, pt:] = pop[i+1, pt:].copy(), pop[i, pt:].copy()

        # мутація
        mutation_mask = np.random.rand(pop_size, bits) < 0.01
        pop = np.logical_xor(pop, mutation_mask).astype(int)

    fitness = np.array([func(decode(ind)) for ind in pop])
    best = pop[np.argmin(fitness)]

    return decode(best), min(fitness)


Алгоритм зграї сірих вовків

In [70]:
def grey_wolf_optimizer(func, bounds, wolves=20, iterations=80):
    a, b = bounds
    pop = np.random.uniform(a, b, wolves)

    for t in range(iterations):

        fitness = np.array([func(x) for x in pop])
        idx = np.argsort(fitness)

        alpha, beta, delta = pop[idx[:3]]

        A = 2 - 2*t/iterations

        for i in range(wolves):
            r = np.random.rand(6)

            A1, C1 = 2*A*r[0]-A, 2*r[1]
            A2, C2 = 2*A*r[2]-A, 2*r[3]
            A3, C3 = 2*A*r[4]-A, 2*r[5]

            D1 = abs(C1*alpha - pop[i])
            D2 = abs(C2*beta  - pop[i])
            D3 = abs(C3*delta - pop[i])

            pop[i] = (alpha - A1*D1 +
                      beta  - A2*D2 +
                      delta - A3*D3) / 3

        pop = np.clip(pop, a, b)

    fitness = np.array([func(x) for x in pop])
    best_idx = np.argmin(fitness)

    return pop[best_idx], fitness[best_idx]


Тестування описаних вище
алгоритмів (генетичний алгоритм та алгоритм зграї сірих вовків) на
одновимірній тестовій одноекстремальній функції.

In [ ]:
def f1(x):
    return x**2

def f2(x):
    return  x**2 * (3 - x)**4 * np.sin(12 * np.pi * x)

def f3(x):
    return x**3 * (3 - x)**5 * np.cos(10 * np.pi * x)

#Параметричні
def f4(x):
    return (0 - x)**1 * (3 - x)**1 * np.sin(6 * np.pi * x) * np.sin(8 * np.pi * x)



In [ ]:
# Правильні межі для кожної функції
functions_with_bounds = [
    (f1, "f1: x²", (-5, 5)),                    # парабола - будь-які, а всі інші тільки [0, 3]
    (f2, "f2: x²(3-x)⁴sin(12πx)", (0, 3)),    
    (f3, "f3: x³(3-x)⁵cos(10πx)", (0, 3)),      
    (f4, "f4: (0-x)(3-x)sin(6πx)sin(8πx)", (0, 3))  
]

for func, name, bounds in functions_with_bounds:
    print(f"\nФункція: {name}")
    
    x_ga, val_ga = genetic_algorithm(func, bounds)
    print(f"GA  -> x = {x_ga:.6f}, f(x) = {val_ga:.6f}")
    
    x_gwo, val_gwo = grey_wolf_optimizer(func, bounds)
    print(f"GWO -> x = {x_gwo:.6f}, f(x) = {val_gwo:.6f}")


Функція: f1: x²
GA  -> x = -0.000076, f(x) = 0.000000
GWO -> x = 0.000000, f(x) = 0.000000

Функція: f2: x²(3-x)⁴sin(12πx)
GA  -> x = 0.792081, f(x) = -14.907978
GWO -> x = 0.958329, f(x) = -15.957774

Функція: f3: x³(3-x)⁵cos(10πx)
GA  -> x = 1.500023, f(x) = -25.628117
GWO -> x = 1.100527, f(x) = -32.954007

Функція: f4: (0-x)(3-x)sin(6πx)sin(8πx)
GA  -> x = 1.929183, f(x) = -1.964854
GWO -> x = 0.930731, f(x) = -1.831818


перевірка

In [157]:
def check_functions():
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    funcs = [f1, f2, f3, f4]
    names = ["f1: x²", "f2: x²(3-x)⁴sin(12πx)", "f3: x³(3-x)⁵cos(10πx)", "f4: (0-x)(3-x)sin(6πx)sin(8πx)"]
    bounds_list = [(-5, 5), (0, 3), (0, 3), (0, 3)]
    
    for i, (func, name, bounds) in enumerate(zip(funcs, names, bounds_list)):
        ax = axes[i//2, i%2]
        x = np.linspace(bounds[0], bounds[1], 1000)
        y = func(x)
        
        ax.plot(x, y, 'b-', linewidth=1.5)
        ax.set_xlabel('x')
        ax.set_ylabel('f(x)')
        ax.set_title(f"{name}\nМежі: {bounds}")
        ax.grid(True, alpha=0.3)
        
        min_idx = np.argmin(y)
        ax.plot(x[min_idx], y[min_idx], 'ro', markersize=8, 
                label=f'Мін: x={x[min_idx]:.3f}, f={y[min_idx]:.3f}')
        ax.legend()
    
    plt.tight_layout()
    plt.show()

check_functions()

----

### Task 4

Процес пошуку
глобального екстремуму. Тобто потрібно продемонструвати положення
популяції на функції на кожній ітерації.

In [ ]:
def ga_visual_like_image(func, bounds, bits=16, pop_size=50, generations=30):
    a, b = bounds

    def decode(ind):
        return a + int("".join(map(str, ind)), 2) / (2**bits - 1) * (b - a)

    # Початкова популяція
    pop = np.random.randint(0, 2, (pop_size, bits))
    decoded = np.array([decode(ind) for ind in pop])
    fitness = np.array([func(xi) for xi in decoded])

    sort_idx = np.argsort(fitness)
    fitness = fitness[sort_idx]
    decoded = decoded[sort_idx]
    pop = pop[sort_idx]

    best_hist = []
    P = 0.7

    
    fig, axs = plt.subplots(3, 1, figsize=(8, 8))  

    # 1. Функція + популяція 
    x = np.linspace(a, b, 10000)
    y = func(x)
    axs[0].plot(x, y, 'b-', lw=1.5)

    scat = axs[0].scatter(decoded, fitness, c="red", s=80, marker='o')

    axs[0].set_xlim(a, b)
    axs[0].set_ylim(-0.5, 0.5)
    axs[0].set_title(f"Покоління 0, Найкраще = {fitness[0]:.4f}")
    axs[0].grid(True, alpha=0.3)

    # 2. Стовпчикова діаграма
    bars = axs[1].bar(
        np.arange(pop_size),
        fitness,
        width=0.8
    )

    axs[1].set_xlim(0, pop_size)
    axs[1].set_ylim(-0.5, 0.5)
    axs[1].set_xlabel('Особини')
    axs[1].set_ylabel('Фітнес')
    axs[1].set_title('Розподіл значень у популяції')
    axs[1].grid(True, alpha=0.3, axis='y')

    # 3. Графік збіжності 
    conv_line, = axs[2].plot([], [], linewidth=2)
    axs[2].set_xlim(1, generations)
    axs[2].set_ylim(-0.5, 0.5)
    axs[2].set_xlabel('Покоління')
    axs[2].set_ylabel('Найкраще значення')
    axs[2].set_title('Збіжність генетичного алгоритму')
    axs[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show(block=False)


    for gen in range(1, generations + 1):


        i1 = np.random.randint(0, pop_size)
        i2 = np.random.randint(0, pop_size)

        child_size = np.random.randint(pop_size, 2 * pop_size)
        child_pop = np.zeros((child_size, bits), dtype=int)

        for i in range(child_size):
            cross_point = np.random.randint(1, bits)
            if np.random.rand() < 0.5:
                child_pop[i] = np.concatenate([pop[i1, :cross_point],
                                               pop[i2, cross_point:]])
            else:
                child_pop[i] = np.concatenate([pop[i2, :cross_point],
                                               pop[i1, cross_point:]])

        
        for i in range(child_size):
            if np.random.rand() < P:
                cross_point = np.random.randint(0, bits)
                child_pop[i, cross_point] = 1 - child_pop[i, cross_point]

        #об'єднання
        combined_pop = np.vstack([pop, child_pop])
        combined_decoded = np.array([decode(ind) for ind in combined_pop])
        combined_fitness = np.array([func(xi) for xi in combined_decoded])

        sort_idx = np.argsort(combined_fitness)
        combined_fitness = combined_fitness[sort_idx]
        combined_decoded = combined_decoded[sort_idx]
        combined_pop = combined_pop[sort_idx]

        pop = combined_pop[:pop_size]
        decoded = combined_decoded[:pop_size]
        fitness = combined_fitness[:pop_size]

        best_hist.append(fitness[0])

        # оновлення графіків
        scat.set_offsets(np.c_[decoded, fitness])
        axs[0].set_title(f"Покоління {gen}, Найкраще = {fitness[0]:.4f}")

        for i, bar in enumerate(bars):
            bar.set_height(fitness[i])

        conv_line.set_data(range(1, gen + 1), best_hist)

        fig.canvas.draw()
        fig.canvas.flush_events()
        time.sleep(0.15)

    plt.show()


In [160]:
def f(x):
    return x * (1 - x) * np.sin(15 * np.pi * x)


ga_visual_like_image(f, (0, 1))

----

### Task 5

Випадок пошуку глобального екстремуму
двовимірної (багатовимірної) функції. Також показати процес пошуку
глобального екстремуму. Тобто потрібно продемонструвати положення
популяції на функції на кожній ітерації. Показати зміну положення
популяції на контурному графіку для двовимірних функцій.

In [223]:
def ga_2d_animation_min(func, bounds,
                    pop_size=40,
                    generations=30,
                    crossover_p=0.8,
                    mutation_p=0.25):

    (xmin, xmax), (ymin, ymax) = bounds

    pop = np.column_stack((
        np.random.uniform(xmin, xmax, pop_size),
        np.random.uniform(ymin, ymax, pop_size)
    ))

    x = np.linspace(xmin, xmax, 400)
    y = np.linspace(ymin, ymax, 400)
    X, Y = np.meshgrid(x, y)
    Z = func(X, Y)

    plt.ion()
    fig = plt.figure(figsize=(12, 5))

    # 3D поверхня
    ax1 = fig.add_subplot(1, 2, 1, projection="3d")
    ax1.plot_surface(X, Y, Z, cmap="viridis", alpha=0.7)
    scat3d = ax1.scatter(pop[:, 0], pop[:, 1],
                         func(pop[:, 0], pop[:, 1]),
                         c="red", s=40)

    ax1.set_xlabel("x")
    ax1.set_ylabel("y")
    ax1.set_zlabel("f(x,y)")

    #Контурний графік
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.contour(X, Y, Z, 40, cmap="viridis")
    scat2d = ax2.scatter(pop[:, 0], pop[:, 1],
                         c="red", s=50, edgecolors="black")

    ax2.set_aspect("equal")

    plt.tight_layout()

    #  Основний цикл 
    for gen in range(1, generations + 1):

        fitness = func(pop[:, 0], pop[:, 1])

        i1 = np.random.randint(0, pop_size, pop_size)
        i2 = np.random.randint(0, pop_size, pop_size)

        mask = (fitness[i1] < fitness[i2])[:, None]
        selected = np.where(mask, pop[i1], pop[i2])

        # Схрещування
        new_pop = selected.copy()
        for i in range(0, pop_size - 1, 2):
            if np.random.rand() < crossover_p:
                alpha = np.random.rand()
                p1, p2 = new_pop[i], new_pop[i + 1]
                new_pop[i]     = alpha * p1 + (1 - alpha) * p2
                new_pop[i + 1] = alpha * p2 + (1 - alpha) * p1

        # Мутація
        for i in range(pop_size):
            if np.random.rand() < mutation_p:
                new_pop[i] += np.random.normal(0, 0.15, size=2)

        # Обмеження
        new_pop[:, 0] = np.clip(new_pop[:, 0], xmin, xmax)
        new_pop[:, 1] = np.clip(new_pop[:, 1], ymin, ymax)

        pop = new_pop

        # Анімація 
        scat2d.set_offsets(pop)
        scat3d._offsets3d = (
            pop[:, 0],
            pop[:, 1],
            func(pop[:, 0], pop[:, 1])
        )

        best = np.min(func(pop[:, 0], pop[:, 1]))
        ax1.set_title(f"3D | Покоління {gen}")
        ax2.set_title(f"Контури | Покоління {gen}, best = {best:.4f}")

        fig.canvas.draw()
        fig.canvas.flush_events()
        time.sleep(0.12)

    plt.ioff()
    plt.show()

    fitness = func(pop[:, 0], pop[:, 1])
    idx = np.argmin(fitness)

    return pop[idx], fitness[idx]

Функція Ізома

In [224]:
def easom(x, y): 
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))

best_point, best_value = ga_2d_animation_min(
    easom,  
    bounds=((-10, 10), (-10, 10)),
    pop_size=100,
    generations=30
)

print(f"x* = {best_point[0]:.4f}, y* = {best_point[1]:.4f}")
print(f"f(x*, y*) = {best_value:.6f}")


x* = 3.1423, y* = 3.1418
f(x*, y*) = -0.999999


Функція Растринга

In [217]:
def rastrigin(x, y, A=10):
    return (
        2 * A
        + (x**2 - A * np.cos(2 * np.pi * x))
        + (y**2 - A * np.cos(2 * np.pi * y))
    )

best_point, best_value = ga_2d_animation_min(
    rastrigin,
    bounds=((-5.12, 5.12), (-5.12, 5.12)),
    pop_size=120,
    generations=50
)


print(f"x* = {best_point[0]:.4f}, y* = {best_point[1]:.4f}")
print(f"f(x*, y*) = {best_value:.6f}")

x* = -0.0001, y* = -0.9949
f(x*, y*) = 0.994962


Функція Ерклі

In [220]:
def ackley(x, y):
    term1 = -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2)))
    term2 = -np.exp(0.5 * (np.cos(2 * np.pi * x) +
                           np.cos(2 * np.pi * y)))
    return term1 + term2 + 20 + np.e


best_point, best_value = ga_2d_animation_min(
    ackley,
    bounds=((-2, 2), (-2, 2)),
    pop_size=50,
    generations=30
)

print(f"x* = {best_point[0]:.4f}, y* = {best_point[1]:.4f}")
print(f"f(x*, y*) = {best_value:.6f}")

x* = 0.0005, y* = -0.0010
f(x*, y*) = 0.003192


Функція «хрест на підносі»

In [225]:
def cross_in_tray(x, y):
    term = np.abs(
        np.sin(x) * np.sin(y) *
        np.exp(np.abs(100 - np.sqrt(x**2 + y**2) / np.pi))
    )
    return -0.0001 * (term + 1)**0.1


best_point, best_value = ga_2d_animation_min(
    cross_in_tray,
    bounds=((-10, 10), (-10, 10)),
    pop_size=100,
    generations=50,
    mutation_p=0.25
)

print(f"x* = {best_point[0]:.5f}, y* = {best_point[1]:.5f}")
print(f"f(x*, y*) = {best_value:.6f}")


x* = 1.34948, y* = -1.34932
f(x*, y*) = -2.062612


Функція «підставка для яєць»

In [226]:
def eggholder(x, y):
    term1 = -(y + 47) * np.sin(
        np.sqrt(np.abs(x + y/2 + 47))
    )
    term2 = -x * np.sin(
        np.sqrt(np.abs(x - (y + 47)))
    )
    return term1 + term2


best_point, best_value = ga_2d_animation_min(
    eggholder,
    bounds=((-512, 512), (-512, 512)),
    pop_size=150,
    generations=50,
    mutation_p=0.3
)

print(f"x* = {best_point[0]:.4f}, y* = {best_point[1]:.4f}")
print(f"f(x*, y*) = {best_value:.6f}")


x* = 324.0554, y* = -145.1303
f(x*, y*) = -419.583659


Таблична функція Хольдера

In [228]:
def holder_table(x, y):
    term = np.sin(x) * np.cos(y) * np.exp(
        np.abs(1 - np.sqrt(x**2 + y**2) / np.pi)
    )
    return -np.abs(term)


best_point, best_value = ga_2d_animation_min(
    holder_table,
    bounds=((-10, 10), (-10, 10)),
    pop_size=100,
    generations=50,
    mutation_p=0.25
)

print(f"x* = {best_point[0]:.5f}, y* = {best_point[1]:.5f}")
print(f"f(x*, y*) = {best_value:.6f}")


x* = -4.90115, y* = -6.53249
f(x*, y*) = -4.712458


Функція Шаффера 1

In [230]:
def schaffer_f1(x, y):
    num = np.sin(x**2 - y**2)**2 - 0.5
    denom = (1 + 0.001*(x**2 + y**2))**2
    return 0.5 + num / denom

best_point, best_value = ga_2d_animation_min(
    schaffer_f1,
    bounds=((-100, 100), (-100, 100)),
    pop_size=100,
    generations=20,
    mutation_p=0.25
)

print(f"x* = {best_point[0]:.5f}, y* = {best_point[1]:.5f}")
print(f"f(x*, y*) = {best_value:.6f}")


x* = -5.72106, y* = 3.72678
f(x*, y*) = 0.043608


Функція Шаффера 2

In [ ]:
def schaffer2(x, y):
    term1 = 0.5
    term2_num = np.cos(np.sin(np.abs(x**2 - y**2)))**2 - 0.5
    term2_den = (1 + 0.001 * (x**2 + y**2))**2
    return term1 + (term2_num / term2_den)

best_point, best_value = ga_2d_animation_min(
    schaffer2,
    bounds=((-100, 100), (-100, 100)),
    pop_size=150,
    generations=15,
    mutation_p=0.2
)

print(f"x* = {best_point[0]:.6f}, y* = {best_point[1]:.6f}")
print(f"f(x*, y*) = {best_value:.6f}")

x* = 0.081812, y* = 1.225621
f(x*, y*) = 0.295128
Теоретичний мінімум: (0, 1.25313), f = 0.292579


Task7 Додаткова функція*

In [234]:
def three_hump_camel(x, y):
    return 2*x**2 - 1.05*x**4 + (x**6)/6 + x*y + y**2

best_point, best_value = ga_2d_animation_min(
    three_hump_camel,
    bounds=((-5,5), (-5,5)),
    pop_size=80,
    generations=20,
    mutation_p=0.2
)

print(f"x* = {best_point[0]:.5f}, y* = {best_point[1]:.5f}")
print(f"f(x*, y*) = {best_value:.6f}")


x* = 0.00014, y* = 0.00121
f(x*, y*) = 0.000002


----

### Task 6

In [135]:
def rastrigin(x):
    n = x.size
    return 10*n + np.sum(x**2 - 10*np.cos(2*np.pi*x))

In [153]:
def ga_rastrigin_with_required_plots(
        n=5,
        PopSize=20,
        S=30,
        MaxIter=40,
        bounds=(-5.12, 5.12),
        crossover_p=0.8,
        mutation_p=0.2,
        sigma=0.1):

    xmin, xmax = bounds
    f_opt = 0.0

    #  початкова популяція
    pop = np.random.uniform(xmin, xmax, (PopSize, n))

    best_dist_history = []

    plt.ion()
    fig, axs = plt.subplots(2, 1, figsize=(6, 8))
    plt.subplots_adjust(hspace=0.4)
    
    # Визначаємо фіксовані межі для осей
    y_min_fitness = 0  
    y_max_fitness = 100  
    
    y_min_convergence = 0.001 
    y_max_convergence = 100  

    for it in range(1, MaxIter + 1):

        # a) схрещення → S нащадків
        children = np.zeros((S, n))
        for i in range(S):
            p1, p2 = pop[np.random.randint(PopSize, size=2)]
            alpha = np.random.rand()
            children[i] = alpha*p1 + (1-alpha)*p2

        #  б) мутація
        for i in range(S):
            if np.random.rand() < mutation_p:
                children[i] += np.random.normal(0, sigma, n)

        #в) об'єднання
        combined = np.vstack([pop, children])
        combined = np.clip(combined, xmin, xmax)

        fitness = np.array([rastrigin(ind) for ind in combined])

        # г) відбір PopSize кращих
        idx = np.argsort(fitness)
        pop = combined[idx[:PopSize]]
        best_fitness = fitness[idx[:PopSize]]

        #  д) сортування для гістограми
        best_fitness_sorted = np.sort(best_fitness)

        #ж) візуалізація
        axs[0].cla()
        axs[0].bar(
            np.arange(1, PopSize + 1),
            best_fitness_sorted
        )
          
        
        axs[0].set_ylim([y_min_fitness, y_max_fitness])
        axs[0].set_xlim([0, PopSize + 1])

        axs[0].set_title(f"Пристосованість популяції (ітерація {it})")
        axs[0].set_xlabel("Елемент популяції")
        axs[0].set_ylabel("Значення ЦФ")
        axs[0].grid(True, axis="y", alpha=0.3)

        best_dist_history.append(abs(best_fitness_sorted[0] - f_opt))

        axs[1].cla()
        axs[1].plot(best_dist_history, linewidth=2)
        axs[1].set_xlabel("Ітерація")
        axs[1].set_ylabel("| f_best − f* |")
        axs[1].set_title("Збіжність до глобального мінімуму")
        axs[1].grid(True, alpha=0.3)
        
        axs[1].set_ylim([y_min_convergence, y_max_convergence])
        axs[1].set_xlim([0, MaxIter])

        fig.canvas.draw()
        fig.canvas.flush_events()
        time.sleep(0.3)

    plt.ioff()
    plt.show()

In [156]:
ga_rastrigin_with_required_plots(
    n=5,
    PopSize=20,
    S=30,
    MaxIter=30
)